In [1]:
class Node (object):
    def __init__(self, inbound_nodes=[]):
        self.inbound_nodes = inbound_nodes
        self.outbound_nodes = []
        
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        self.value = None
        
    def forward(self):
        raise NotImplemented

In [2]:
class Input(Node):
    def __init__(self):
        Node.__init__(self)
    
    def forward(self, value=None):
        if value is not None:
            self.value = value

In [3]:
class Add(Node):
    def __init__(self, l):
        Node.__init__(self, l)
        self.value=None
        
    def forward(self):
        sum = 0
        for n in self.inbound_nodes:
            sum+=n.value
        self.value = sum

In [11]:
class Mul(Node):
    def __init__(self, l):
        Node.__init__(self, l)
        self.value=None
        
    def forward(self):
        sum = 1
        for n in self.inbound_nodes:
            sum*=n.value
        self.value = sum

In [12]:
def topological_sort(feed_dict):
    input_nodes = [n for n in feed_dict.keys()]
    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
            for m in n.outbound_nodes:
                if m not in G:
                    G[m] = {'in': set(), 'out':set()}
                G[n]['out'].add(m)
                G[m]['in'].add(n)
                nodes.append(m)
    #print("G.elements",G.items())
    L = []
    S=set(input_nodes)
    while(len(S) > 0):
        n = S.pop()
        
        if isinstance(n, Input):
            n.value = feed_dict[n]
        L.append(n)
        for m in n.outbound_nodes:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']) == 0:
                S.add(m)
    #print('L', L)
    return L

In [13]:
def forward_pass(ouput_node, sorted_nodes):
    for n in sorted_nodes:
        n.forward()
    return ouput_node.value

In [14]:
x, y, z = Input(), Input(), Input()
l = [x, y,z]
f = Add(l)
m = Mul(l)
feed_dict = {x:10, y:20, z:30}

sorted_nodes = topological_sort(feed_dict = feed_dict)
output = forward_pass(f, sorted_nodes)
outputm = forward_pass(m, sorted_nodes)

print("According to the miniflow output is {} {}".format(output, outputm))

According to the miniflow output is 60 6000
